In [1]:
import os

In [4]:
%pwd
os.chdir('../')
%pwd

'd:\\MLOps Projects\\data-science-project-with-deployment'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from src.mlopsproject.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARMAS_FILE_PATH # every thing from file gets imported 
from src.mlopsproject.utils.common import read_yaml, create_directories
from src.mlopsproject import logger

In [33]:
class ConfigurationManager:
    def __init__(self,
                config_file_path=CONFIG_FILE_PATH,
                schema_file_path=SCHEMA_FILE_PATH,
                params_file_path=PARMAS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [16]:
import urllib.request as request
import zipfile

In [37]:
# data ingestion component

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    # downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already downloaded")
        
    def extract_zip_file(self):
        """
        zip_file_path = str
        extracts the zip file into unzip directory
        function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion =DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-12 16:35:48,881: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-09-12 16:35:48,886: common: yaml file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-09-12 16:35:48,889: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-09-12 16:35:48,890: common: created directory at: artifacts]
[2025-09-12 16:35:48,891: common: created directory at: artifacts/data_ingestion]
[2025-09-12 16:35:48,891: 2973859658: File already downloaded]
